##Imports

In [ ]:
# !pip install GPUtil
# !pip install numba
# !pip install torch
# import torch
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda
# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()
#     torch.cuda.empty_cache()
#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)
#     print("GPU Usage after emptying the cache")
#     gpu_usage()
# free_gpu_cache()

In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 74.9 MB/s 
     |████████████████████████████████| 6.5 MB 57.5 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 70.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install pyyaml h5py

In [40]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 4.3 MB/s 


In [41]:

import numpy as np

import random

import os, json

import pandas as pd
pd.options.display.max_colwidth = 200

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

import transformers

from sklearn.metrics import f1_score

from transformers import BertTokenizer, TFBertModel, TrainingArguments, Trainer
from tensorflow.keras import backend as K

from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
%matplotlib inline

import re

import logging
tf.get_logger().setLevel(logging.ERROR)

In [4]:
tf.config.list_physical_devices('TPU')

[]

In [5]:
tf.__version__

'2.8.0'

In [6]:
transformers.__version__

'4.17.0'

##Import English Data

In [7]:
#Import data from Google drive
from google.colab import drive
drive.mount('/content/gdrive')

english_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_English/english_train.csv'
                       )

english_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_English/english_dev.csv'
                       )

english_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_English/english_test.csv'
                       )

Mounted at /content/gdrive


In [8]:
english_train

,text,label
0,"The user doing this is happy to see the gutted version on wikipedia, he is gone. Good job. Dispute is over.65.87.185.73",0
1,"That page was listed a couple of days ago, but I didn't had time to check it. Anyway, I see the problem is moot now.Tizio",0
2,"I saw that, thanks. – (talk)",0
3,Fuck you you bitch ass ho.,1
4,I really hate you \n\nAnd I want to do nasty things to you,1
...,...,...
24465,"""\n\n Don't forget the offense against non-sensible people. (Talk) """,0
24466,This isn't about Protestant IRA members. This is about YOU removing content from wikipedia and applying different set of standards to Irish Republicans than anyone else with a page on wikipedia. Y...,0
24467,"""\n\nDear Azure Citizen:\n I wrote a """"talk"""" comment about a pernicious phenomenon which has infected Wikipedia in general, and this article in particular. I didn't name anyone in particular...",0
24468,I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I...,1


In [9]:
english_dev

,text,label
0,Mortgage lending \n\nIs this the Commerce Bank that owns East-West Mortgage?,0
1,Restore Irish E-Sports page \n\nBased prior decision that this page can not be put up for speedy deletion and agreement with Jerry's decision your deletion of the Irish E-Sports page was reverte...,0
2,"Sorry, I thought no one would read it.",0
3,"You have made five edits today, all were silly and unconstructive and all have been reverted. Stop wasting everyone's time.",1
4,"""\n\nImage copyright problem with Image:AUSEPassport.jpg\n \nThanks for uploading Image:AUSEPassport.jpg. However, the image may soon be deleted unless we can determine the copyright holder and co...",0
...,...,...
3054,Argg!I just did it again!!!! (forgot to sign).,0
3055,"Wendy, get lost. Really. Not everybody shares your bigoted views. And Antonio, quite frankly, you're a fucker for comparing Mercado's sexuality to Babe Ruth's alcoholism. Get the fuck off the site...",1
3056,"""\n\n Hey Jerk \n\n""""You need to read the editnotice"""", well I did stupid. You don't have to be a jerk about it, loser.""",1
3057,Oh that is wonderful how those IP's just change around isn't it? Any other key evidence you'd like to tamper with? It sounds like you've done a lot wrong. It's really just a matter of when it catc...,0


In [10]:
english_test

,text,label
0,"Why are you still here? Can you not find anything more important to do, like killing yourself?",1
1,Its an inaccurate entry. Wikipedia should aim to be as accurate as possible.,0
2,"This bot is seriously driving us crazy\nThis image exists, I must kill it!",1
3,PLEASE STOP VANDALISING MY USER TALK PAGE,0
4,"I've removed the Orwell referrence from the influences section, as its now been a week and no response.",0
...,...,...
3054,u suck at making movies\n\nu pathetic geek\n\nur premise is shaky\n\nur outlook is bleak\n\nthere isnt a too painful way for ur life to end\n\nu really need a girlfriend,1
3055,Dont be gay - Hitler was a form of hero!!,1
3056,I wish to be block from editing Goddamit Goddamit block me from editing!,1
3057,Why did you block my page you twat. I am going to find you and me and the lads are going to play footie with your testacles. My mate told me he'd had some trouble with wankers on wikipedia some ga...,1


##Import Hindi Data

In [11]:
#Import Hindi data from Google drive
from google.colab import drive
drive.mount('/content/gdrive')

hindi_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Hindi_New/hindi_train.csv'
                       )

hindi_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Hindi_New/hindi_dev.csv'
                       )

hindi_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Hindi_New/hindi_test.csv'
                       )

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
hindi_train

,text,label
0,"RT @RishiPrasadOrg: ईश्वर किसीको प्राप्त नहीं होता, ब्रह्म किसीको प्राप्त नहीं होता बल्कि प्राप्त हुआ-सा लगता है।वह भी किसको? जिसको नहीं मि…",0
1,@BBCHindi इसे कहते हैं खतरो का खिलाड़ी।,0
2,७० साल जतन के ७ साल पतन के #IndiaCovidCrisis,0
3,"RT @AagayiNavya: चंपारण में एक चीनी मिल बंद पड़ी थी,२०१४ में मोदी ने कहा था अब यहां तभी आऊंगा जब इसी चीनी मिल की चीनी से चाय पियूंगा...\nआज…",1
4,सिर्फ़ सरकारें बदलती हैं हालात नहीं बदलते l \nकांग्रेस शासित राजस्थान की जेल में फिर एक मुसलमान \nक़ैदी की हत्या (लिंचिंग) l \n कांग्रेस शासन में राजस्थान में तीसरी लिंचिंग हो गई\nकांग्रेस सिर्फ भ...,1
...,...,...
4230,आज़ाद ख़याल वो नहि वो फूहड़ हो आज़ाद ख़याल वो जो हक़ीक़त को #बज्म,0
4231,चलो वक़्त में थोड़ा ठहर के देखे ~~ @niksism6 --&gt;&gt; my thoughts on the YourQuote app at https://t.co/PEVD4AjLw8 --&gt;&gt; Retweet If You Like this।। #niksismwrites✍️ #CoronaPandemic #...,0
4232,"RT @PihuTyagi_: लोगों के काम आते रहिये क्योंकि ,क़ुदरत का एक उसूल है कि...\n\nजिस कुंए से लोग पानी पीते रहे ,वो कभी सूखता नही है ...!!…",0
4233,"1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n17000 सिखो, 5000 भोपाली, 3000 तमिलों का कत्लेआम करवाने वाला भारत रत्न कैसे हो सके है?",1


In [13]:
hindi_dev

,text,label
0,"#ModiKaVaccineJumla जुमला वाला प्रधानमंत्री, जैसी करनी वैसी भरनी। लोग जात पात पर वोट करते हैं तो उसे भोंखने वाला कुत्ता ही मिलेगा या फिर बददिमाग गधा नाकि कोई अच्छा इंसानियत का हुनर रखने वाला नेता।",1
1,Mar javunga aaj me khushi me 😍😍,0
2,"RT @Sonam_Mumbaikar: तेरे इश्क़ में हद से गुजर जाऊं\n\nमैं झूठ कहूं तो, मर जाऊं...😍\n\n@Sonam_Mumbaikar",0
3,@Physicsgaurav Ye sale pde likhe chutiya hai 😜🤘,1
4,@RachnaSinghSP अरे चाची हो चाची.. जब सैफई में रंडी नचा के नकटेढवा देश के बर्बाद करत रहल तब तोहार जुनून कहाँ रहल हो चाची... खालीभर बकलोली करय जानेलू ये चाची.... 🤣🤣🤣🤣,1
...,...,...
524,"RT @qshaikhu: @attorneybharti भाजपा वाले तो अपना टेबल भी लेकर निकल लिए, फ्रूटी की खाली बोटल छोड़कर, स्वच्छ भारत मैं अपना योगदान देकर । htt…",1
525,"RT @BJP4India: भ्रष्टाचार हो, महंगाई हो या 1984 में हजारों सिखों का कत्ल हुआ हो इनका जवाब यही होता है- 'हुआ तो हुआ'।\n\nजिस अहंकार में ये लोग…",1
526,"शुक्रिया इमरान प्रताप गढ़ी साहेब शुक्रिया अमनातुल्लह साहेब ,, इस दुख की घड़ी में ओसामा शहाब का साथ देने के लिए साथ खड़े होने के लिए,,उन लोगों का भी बहुत शुक्रिया जो दिल्ली में ओसामा भाई के साथ खड...",0
527,#bengalviolence Be careful everyone. Some unscrupulous people are posting these pictures on social media to spread tension in the state and the country. #CovidIndia #COVIDEmergencyIndia #ममता_की_...,0


In [14]:
hindi_test

,text,label
0,"@zishanAliRJD @iAnantSingh_ *ओसामा साहब ने सिर्फ उस ट्वीट और ट्विटर अकाउंट को फर्जी बताया है ना कि राजद का पक्ष लिए और ना की राजद का गुणगान किये है, संयम रखिए जल्द ही सब कुछ सामने आएगा।* #JusticeF...",0
1,"@China_Amb_India @narendramodi I am shocked they didn't mention a line (Karam, you have been our worst sufferer) (करम हमारे ख़राब ह भुगत आप रहे ह) #wohanvirus #chinesevirus",0
2,कल से 18 से ऊपर वालो को हवा की वैक्सीन लगेगी... #ModiKaVaccineJumla,0
3,इधर की बात उधर करने में \nआज भी जीमेल से आगे है \nफीमेल!!!,0
4,☯️ मोदी जी ➡️ सोंगंद मुझे इस मिट्टी की मै देश नहीं मिटने दूंगा मै देश नहीं झुकने दूंगा !! और हमने भैंचोद ये बात सीरियस लेली!!!! ले कोरॉना➡️ सालो अब भुगतो 👹 #ModiMustResign #ModiKaVaccineJumla,1
...,...,...
525,"हे प्रभु!! निंदा????? नेताओं की , और सरकारों के निकम्मेपन को छुपाने के लिए एक शब्द काफी है ""निंदा"" । #BengalBurning https://t.co/FcFx0eTSW0",1
526,"अगर आपको पाकिस्तान या बांग्लादेश के हिंदुओं की चिंता होती है की उनके साथ वहा गलत हो रहा है और आप उनको भारत बुलाना चाहते हो, भारत बुला के भी करोगे क्या? भारत के हिंदुओ का घर लूटा जा रहा है, हिंदू म...",1
527,RT @SHIVANI92_: क्या ऐसा करना चाहिए? अपना विचार जरुर कमेंट करें।\n@cp4bhai @MLArajeshSP @ShivaYa36977421 @Narendryadav01 @0359R @AshishS5185…,0
528,RT @pravinj14696933: राजीव ने बोफोर्स खाया हैं\nसोनिया ने खाया था अगस्ता\nवाड्रा ने ज़मीन हड़पी हैं\nराहुल ने हेरॉल्ड डकारा थां\n\nइंदिरा ने इमर…,1


##Import Korean Data

In [15]:
#Import Korean data from Google drive
from google.colab import drive
drive.mount('/content/gdrive')

korean_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Korean_New/korean_train.csv'
                       )

korean_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Korean_New/korean_dev.csv'
                       )

korean_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Korean_New/korean_test.csv'
                       )

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
korean_train

,text,label
0,불쌍해 보이는 이윤 뭘까?~,0
1,독과점의 결과,0
2,별 시덥지않은 악플들은 모냐,0
3,사랑의 불시착 하는시간인데 이상한 노잼 드라마가 하고있다 ㅡㅡ 어딨는거냐 표치수~~~~~,1
4,저밖에몰라..남은사람 어쩌라고. 참 이기적이네,0
...,...,...
5828,다음 생에는 중국 남자 만나야지,0
5829,난 왜 이부진이 생각나는걸까?,0
5830,4년전정준영카톡이 이게 이렇게 큰일이냐?ㅋ 더럽네정말.. 누가봐도 꼬리짜르기구만,1
5831,악플러 쓰레기들 많네,1


In [17]:
korean_dev

,text,label
0,다 그러고 애낳고 키웠고 다 그러고 산다예전에 우리엄마들은 어떻게 애키우며 밭일하고 살았는지...다 본인이 선택한 인생이고 힘든거 이상의 기쁨이 있다기자들도 이런거 퍼날리지말고 발로뛴 기사나 써라,1
1,이제 별 감흥도 없는 애를 멀...,1
2,홍상수 김민희 좋아하는 감독도 배우도 아니지만 남여관계 모르는 거다. 이렇게 비난받을 일은 아닌듯..,0
3,이쁘게 컸네^^,0
4,아니 저 금동현 자리에 진우가 있어야 한다며 ??,0
...,...,...
724,에이프릴 이나은 고정 해도될듯 너무잘하더라♡,0
725,다니엘 잘못한거 없어요 공룡과 힘겨운 분쟁할때당신을 행복하게 해줄 사람 만나는 거ㅡ무조건 응원합니다,0
726,새로운 남자친구나 남편은 저 사이가 얼마나 불편할까,1
727,수아가 불쌍하다 납치되서 20년을 힘들게 살고 남자놈은 수아버리고 민채린한테 가서 다리에서 뛰려고했다 수아가 저둘때메 눈안뒤집히겠냐 그리고 수아정체알고도 부모와못만나게 한채린이를 알게된다면 그집서쫒겨나야죠,0


In [18]:
korean_test

,text,label
0,"팀으로 데뷔한거면 개인활동 했어도 N빵 해야지... 그게 팀을 위해서도 맞는거고~~ 근데 여기 댓글은 다 왜 나누냐는 식이네, 당장 혼자만 배불러야 된다는 심리들인건지, 팀이 유지되야 인기도 오래갈텐데",0
1,왕지혜 34살이지만 외모는 인정한다.여자라면 이정도는 되야지,0
2,여자들이 80프로잉 악플 남자는 여자욕 안해,1
3,설현이 떨고 있다... 아니겠지 아닐거야 그것만은 안돼,0
4,다된 기생충 잔치에재 뿌린 방가방송과 안현모 다신 나서지 말자,1
...,...,...
725,유재석 재벌인디,0
726,보컬만 아니면 상관없잖아..?,0
727,셔누가 무슨 잘뭇했나요? 결백하다...,0
728,사귐 사귄다고 말함되지 참 답답따,0


In [19]:
# # Function to pre-process tweets


# def clean_tweet(tweet):
#   tweet = re.sub(r"\B@\w+", "USERNAME", tweet) #Replace Username
#   tweet = re.sub(r"http\S+", "", tweet) # Remove url

#   hashtags = re.findall(r"(#\w+)", tweet)
#   for hs in hashtags:
#     words = " ".join(ws.segment(hs))
#     if len(words) > 0:
#       tweet = tweet.replace(hs, words)
#     else:
#       tweet = tweet.replace(hs, hs[1:])
  
#   return tweet


##Import Translated Data

In [62]:
#Import Translated data from Google drive
from google.colab import drive
drive.mount('/content/gdrive')

korean_translated_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Translated_Google_Translate/korean_translated_train.csv'
                       )

korean_translated_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/korean_translated_dev.csv'
                       )

korean_translated_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/korean_translated_test.csv'
                       )

hindi_translated_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Translated_Google_Translate/hindi_translated_train.csv'
                       )

hindi_translated_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/hindi_translated_dev.csv'
                       )

hindi_translated_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/hindi_translated_test.csv'
                       )


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [63]:
korean_translated_train

,text,label
0,What is the poor-looking profit?,0
1,result of monopoly,0
2,Are there any malicious comments that aren't too bitter?,0
3,"It's time for a love crash, but a strange no-jam drama is being played ㅡㅡ Where are you? Pyo Chi-soo~~~~~",1
4,I don't know.. What about the rest of me? It's so selfish,0
...,...,...
5828,I need to meet a Chinese man in my next life,0
5829,Why am I thinking of Lee Boo-jin?,0
5830,4 years ago Jung Joon-young Kakao Talk is this such a big deal?,1
5831,There are a lot of malicious comments,1


In [64]:
korean_translated_dev

,text,label
0,"After all, they had children and raised them, and they all live like that. In the past, how my mothers raised children and worked in the fields... They are all their own choices, and there is joy...",1
1,Far from a kid who doesn't have much inspiration anymore...,1
2,"Hong Sang-soo and Kim Min-hee I am neither a favorite director nor an actress, but I do not know the relationship between men and women. I don't think it's something to be criticized for.",0
3,You grew up pretty ^^,0
4,"No, he said that Jinwoo should be in the place of Geum Dong-hyun??",0
...,...,...
724,April's Naeun looked so good that it could be fixed♡,0
725,Daniel didn't do anything wrong To meet someone who will make you happy when you have a difficult dispute with a dinosaur - I support you unconditionally,0
726,How uncomfortable would my new boyfriend or husband be with me?,1
727,"feel sorry for Soo-ah, I was kidnapped and I had a hard time living for 20 years. The man left Su-ah and went to Min Chae-rin to run on the bridge. Won't you turn your eyes when Su-a is gone? it'...",0


In [65]:
korean_translated_test

,text,label
0,"If you debuted as a team, even if you did individual activities, you should be N bread... That's right for the team too~~ But it's like why are you sharing all the comments here, maybe it's the m...",0
1,"Wang Ji-hye is 34 years old, but she admits her looks. If it's a woman, it should be like this",0
2,"Women get 80% of malicious comments, and men don't lust for women.",1
3,Seolhyun is trembling... no no no no no,0
4,"Banga Broadcasting and Ahn Hyun-mo, who sprinkled ashes on the exhausted parasite feast, let's not go out again",1
...,...,...
725,Yoo Jae-seok chaebol indie,0
726,It doesn't matter if it's not vocals...?,0
727,What did Shownu do well? innocent...,0
728,"They say they're dating, so it's frustrating",0


In [66]:
hindi_translated_train

,text,label
0,"RT @RishiPrasadOrg: God is not attained by anyone, Brahman is not attained by anyone but seems to have been achieved. To whom too? Who doesn't get…",0
1,@BBCHindi It is called Khatron Ka Khiladi.,0
2,"70 years of saving, 7 years of downfall #IndiaCovidCrisis",0
3,"RT @AagayiNavya : A sugar mill was closed in Champaran, in 2014 Modi had said that now I will come here only when I will drink tea from the sugar of this sugar mill… today…",1
4,"Only the governments change, the situation does not change. Again a Muslim prisoner was murdered (lynching) in the jail of Congress-ruled Rajasthan. The third lynching took place in Rajasthan unde...",1
...,...,...
4230,"Free thoughts are not those who are stupid, free thoughts are those who destroy reality.",0
4231,Let's stop in time and see ~~ @niksism6 ---&gt;&gt; my thoughts on the YourQuote Retweet If You Like this.. #niksismwrites✍️ #CoronaPandemic #Bazm #Shayari #IndiaCovidCrisis https://t.co/PAedmxvIZo,0
4232,"RT @PihuTyagi_: Keep coming to people's work because, there is a principle of nature that... the well from which people keep drinking water, it never dries up...!!…",0
4233,"Godse could be a terrorist by killing 1 man, so how could the one who killed 17000 Sikhs, 5000 Bhopalis, 3000 Tamils become Bharat Ratna?",1


In [67]:
hindi_translated_dev

,text,label
0,"#ModiKaVaccineJumla Prime Minister with a jumla, do whatever you want to do. If people vote on the basis of caste, then he will get only a barking dog or a bad minded donkey and not a leader with...",1
1,Mar javunga aaj me khushi me,0
2,"RT @Sonam_Mumbaikar: If I tell lies, I will go beyond my limits...😍 @Sonam_Mumbaikar",0
3,@Physicsgaurav Ye sale pde likhe chutiya hai,1
4,"@RachnaSinghSP Hey aunty ho aunty.. when you are ruining the country of Randi Nacha in Saifai, then where is your passion?",1
...,...,...
524,"RT @qshaikhu : @attorneybharti BJP people even took their tables and left empty bottles of frutti, contributing to clean India. htt…",1
525,"RT @BJP4India: Be it corruption, inflation or the killing of thousands of Sikhs in 1984, the answer is 'hua to hua'. The arrogance in which these people…",1
526,"Thank you Imran Pratap Gadhi Saheb Thank you Amnatollah Saheb, For standing together to support Osama Shahab in this sad hour, Many thanks also to those who stood with Osama Bhai in Delhi, Bihar ...",0
527,#bengalviolence Be careful everyone. Some unscrupulous people are posting these pictures on social media to spread tension in the state and the country. #CovidIndia #COVIDEmergencyIndia #Mamata_Ki...,0


In [68]:
hindi_translated_test

,text,label
0,"@zishanAliRJD @iAnantSingh_ *Osama sahib has just called that tweet and twitter account fake, not favoring RJD and not praising RJD, have patience, everything will come out soon.* #JusticeForShah...",0
1,"@China_Amb_India @narendramodi I am shocked they didn't mention a line (Karam, you have been our worst sufferer) (Karam, you have been our worst sufferer) #wohanvirus #chinesevirus",0
2,"From tomorrow, people above 18 will get air vaccine... #ModiKaVaccineJumla",0
3,"Even today, female is ahead of Gmail in talking about it here and there!!!",0
4,"️ Modi ji ️ Songand I will not allow the country to be destroyed by this soil, I will not let the country bow down !! And we took this thing seriously!!!! Le Corona ️ Years now suffer #ModiMustRes...",1
...,...,...
525,Oh God!! Condemnation????? One word is enough to hide the inefficiency of politicians and governments. #BengalBurning https://t.co/FcFx0eTSW0,1
526,"If you are worried about the Hindus of Pakistan or Bangladesh that there is wrong with them and you want to call them to India, what will you do by calling them to India? The houses of Hindus of I...",1
527,RT @SHIVANI92_: Should it be done? Do comment your thoughts. @cp4bhai @MLArajeshSP @ShivaYa36977421 @Narendryadav01 @0359R @AshishS5185…,0
528,"RT @pravinj14696933: Rajiv has eaten Bofors, Sonia has eaten Agusta Vadra has grabbed land, Rahul has Harold Dakara, Indira has Immar…",1


##Tokenize Text

In [20]:
# Specify tokenzer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
mbert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [21]:
#Tokenize text, see next cell for choice of max_length
max_length = 512



# English
english_x_train = tokenizer(list(english_train.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
english_y_train = english_train.label

english_x_dev = tokenizer(list(english_dev.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
english_y_dev = english_dev.label

english_x_test = tokenizer(list(english_test.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
english_y_test = english_test.label

In [22]:
# Max number Non zero elements in input_ids, i.e. longest tokenized sentence for each set, max length can be *just* more than this
# max tokenized length is 512 :( )
# Choosing 512 in cell above

print(np.apply_along_axis(np.count_nonzero, 1, english_x_train.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, english_x_dev.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, english_x_test.input_ids.numpy()).max())

512
512
512


In [23]:
#Tokenize Hindi text, see next cell for choice of max_length
max_length = 512



#Hindi
hindi_x_train = tokenizer(list(hindi_train.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_y_train = hindi_train.label

hindi_x_dev = tokenizer(list(hindi_dev.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_y_dev = hindi_dev.label

hindi_x_test = tokenizer(list(hindi_test.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_y_test = hindi_test.label

In [24]:
# Max number Non zero elements in input_ids, i.e. longest tokenized sentence for each set, max olength can be *just* more than this
# Choosing 225 in cell above

print(np.apply_along_axis(np.count_nonzero, 1, hindi_x_train.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, hindi_x_dev.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, hindi_x_test.input_ids.numpy()).max())

220
170
181


In [25]:
#Tokenize Hindi text, see next cell for choice of max_length
max_length = 512



#Korean
korean_x_train = tokenizer(list(korean_train.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_y_train = korean_train.label

korean_x_dev = tokenizer(list(korean_dev.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_y_dev = korean_dev.label

korean_x_test = tokenizer(list(korean_test.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_y_test = korean_test.label

In [26]:
# Max number Non zero elements in input_ids, i.e. longest tokenized sentence for each set, max olength can be *just* more than this
# Choosing 225 in cell above

print(np.apply_along_axis(np.count_nonzero, 1, korean_x_train.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, korean_x_dev.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, korean_x_test.input_ids.numpy()).max())

94
95
91


##Tokenize Translations

In [69]:
#Tokenize Hindi text, see next cell for choice of max_length
max_length = 512



#Hindi
hindi_translated_x_train = tokenizer(list(hindi_translated_train.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_translated_y_train = hindi_translated_train.label

hindi_translated_x_dev = tokenizer(list(hindi_translated_dev.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_translated_y_dev = hindi_translated_dev.label

hindi_translated_x_test = tokenizer(list(hindi_translated_test.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_translated_y_test = hindi_translated_test.label

In [70]:
# Max number Non zero elements in input_ids, i.e. longest tokenized sentence for each set, max olength can be *just* more than this
# Choosing 225 in cell above

print(np.apply_along_axis(np.count_nonzero, 1, hindi_translated_x_train.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, hindi_translated_x_dev.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, hindi_translated_x_test.input_ids.numpy()).max())

180
172
171


In [71]:
#Tokenize Korean Translated text, see next cell for choice of max_length
max_length = 512

#Korean
korean_translated_x_train = tokenizer(list(korean_translated_train.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_translated_y_train = korean_translated_train.label

korean_translated_x_dev = tokenizer(list(korean_translated_dev.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_translated_y_dev = korean_translated_dev.label

korean_translated_x_test = tokenizer(list(korean_translated_test.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_translated_y_test = korean_translated_test.label

In [72]:
# Max number Non zero elements in input_ids, i.e. longest tokenized sentence for each set, max olength can be *just* more than this
# Choosing 225 in cell above

print(np.apply_along_axis(np.count_nonzero, 1, korean_translated_x_train.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, korean_translated_x_dev.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, korean_translated_x_test.input_ids.numpy()).max())

101
90
101


##Create Model Related Functions

In [73]:
def create_classification_model(hidden_sizes = (256, 128), 
                                train_layers = -1, 
                                optimizer=tf.keras.optimizers.Adam(),
                                hidden_initializer = 'he_uniform',
                                dropout = 0):
    """
    Build a simple classification model with mBERT.
    """

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}


    #restrict training to the train_layers outer transformer layers
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in mbert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False


    bert_out = mbert_model(bert_inputs)


    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])


    hidden = tf.keras.layers.Dense(hidden_sizes[0], name='hidden_layer', activation='relu', kernel_initializer = hidden_initializer)(classification_token)
    hidden = tf.keras.layers.LayerNormalization()(hidden)

    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    hidden = tf.keras.layers.Dense(hidden_sizes[1], name='hidden_layer_2', activation='relu')(hidden)
    hidden = tf.keras.layers.LayerNormalization()(hidden)

    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    classification_model.compile(optimizer=optimizer,
                            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                            metrics=['acc',f1_m,precision_m, recall_m])


    return classification_model

In [74]:
# Custom Metrics
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [75]:
# Early stopping callback
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_f1_m', 
                                                 mode = 'max',
                                                 patience=3, 
                                                 restore_best_weights=True)

In [76]:
#Function to add arecord to a json file
def append_record(record):
    with open('/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English 22_3_31/settings_history.txt', 'a') as f:
        json.dump(record, f)
        f.write(os.linesep)

##Fit English

In [47]:
#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rates = [.001]
dropouts = [0, 0.1, 0.2]
hidden_sizes = [(256, 128)]
train_layers = [0, 4]


count = 0
for learning_rate in learning_rates:
  for dropout in dropouts:
    for train_layer in train_layers:
      for hidden_size in hidden_sizes:

        count += 1

        if count<=3:
          continue



        settings = {'count':count,
                    'learning_rate':learning_rate,
                    'dropout':dropout,
                    'train_layer':train_layer,
                    'hidden_size':hidden_size
                    }

        #Define checkpoint directory
        checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English 22_3_31/training_" + str(count) + "/cp.ckpt"
        checkpoint_dir = os.path.dirname(checkpoint_path)

        classification_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )
        
        
        # Create a callback that saves the model's weights
        cp_callback = [early_stopper, tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=0)]

        # Train the model with the new callback
        print('COUNT = {:.0f}'.format(count))
        history = classification_model.fit([english_x_train.input_ids, english_x_train.token_type_ids, english_x_train.attention_mask],
                          english_y_train,
                          validation_data=([english_x_dev.input_ids, english_x_dev.token_type_ids, english_x_dev.attention_mask],
                          english_y_dev),
                                epochs=3,
                                batch_size=20,
                                callbacks=cp_callback,
                                verbose = 1
                                )
        

        append_record({'setings':settings, 'history':history.history})


      

COUNT = 1
Epoch 1/3


ValueError: ignored

In [ ]:
classification_model.predict([english_x_dev.input_ids[:10], english_x_dev.token_type_ids[:10], english_x_dev.attention_mask[:10]])

#Reload Model trained on English (Extra)

In [ ]:
#Define checkpoint directory
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks (1)/Models_English_Pretrain_Hindi_FineTune/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# reload saved model

#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rates = [.001]
dropouts = [0]
hidden_sizes = [(256, 128)]
train_layers = [0]

count = 0
for learning_rate in learning_rates:
  for dropout in dropouts:
    for train_layer in train_layers:
      for hidden_size in hidden_sizes:

        count += 1

        reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                            train_layers = train_layer,
                                                            hidden_initializer = 'he_uniform',
                                                            dropout = dropout,
                                                            hidden_sizes = hidden_size
                                                            )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)


In [ ]:
reloaded_model.evaluate(x=[english_x_dev.input_ids, english_x_dev.token_type_ids, english_x_dev.attention_mask], 
                        y=english_y_dev, 
                        batch_size = 20)

153/153 [==============================] - 77s 470ms/step - loss: 0.3567 - acc: 0.8500 - f1_m: 0.8409 - precision_m: 0.8507 - recall_m: 0.8437


[0.35671889781951904,
 0.8499509692192078,
 0.8409140706062317,
 0.8506607413291931,
 0.8436879515647888]

In [ ]:
reloaded_model.evaluate(x=[hindi_x_dev.input_ids, hindi_x_dev.token_type_ids, hindi_x_dev.attention_mask], 
                        y=hindi_y_dev, 
                        batch_size = 20)

27/27 [==============================] - 17s 459ms/step - loss: 0.6458 - acc: 0.6427 - f1_m: 0.7127 - precision_m: 0.5918 - recall_m: 0.9147


[0.6457741856575012,
 0.6427221298217773,
 0.7127432227134705,
 0.5917838215827942,
 0.9146944880485535]

In [ ]:
# Early stopping callback
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_f1_m', 
                                                 mode = 'max',
                                                 patience=3, 
                                                 restore_best_weights=True)

cp_callback_retrain = [early_stopper]

reloaded_model.fit([hindi_x_train.input_ids, hindi_x_train.token_type_ids, hindi_x_train.attention_mask],
                          hindi_y_train,
                          validation_data=([hindi_x_dev.input_ids, hindi_x_dev.token_type_ids, hindi_x_dev.attention_mask],
                          hindi_y_dev),
                                epochs=15,
                                batch_size=20,
                                callbacks=cp_callback_retrain,
                                verbose = 1
                                )

Epoch 1/15
212/212 [==============================] - 134s 605ms/step - loss: 0.6794 - acc: 0.5750 - f1_m: 0.5391 - precision_m: 0.5801 - recall_m: 0.5477 - val_loss: 0.6601 - val_acc: 0.6163 - val_f1_m: 0.5643 - val_precision_m: 0.6600 - val_recall_m: 0.5172
Epoch 2/15
212/212 [==============================] - 126s 597ms/step - loss: 0.6570 - acc: 0.6054 - f1_m: 0.5787 - precision_m: 0.6085 - recall_m: 0.5866 - val_loss: 0.6626 - val_acc: 0.6125 - val_f1_m: 0.5190 - val_precision_m: 0.7309 - val_recall_m: 0.4238
Epoch 3/15
212/212 [==============================] - 127s 599ms/step - loss: 0.6479 - acc: 0.6260 - f1_m: 0.6004 - precision_m: 0.6264 - recall_m: 0.6172 - val_loss: 0.6286 - val_acc: 0.6484 - val_f1_m: 0.6271 - val_precision_m: 0.6988 - val_recall_m: 0.5997
Epoch 4/15
212/212 [==============================] - 126s 597ms/step - loss: 0.6315 - acc: 0.6390 - f1_m: 0.6231 - precision_m: 0.6486 - recall_m: 0.6423 - val_loss: 0.6365 - val_acc: 0.6465 - val_f1_m: 0.5810 - val_pre

#Run hindi on pretrained English Models


In [ ]:
#Function to add arecord to a json file
def append_record(record):
    with open('/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English_Pretrain_Hindi_FineTune/settings_history.txt', 'a') as f:
        json.dump(record, f)
        f.write(os.linesep)

In [ ]:
for i in range(2,4):

  #Define reload directory
  reload_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English 22_3_31/training_" + str(i) + "_Good/cp.ckpt"
  reload_dir = os.path.dirname(reload_path)

  #Define checkpoint directory
  checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English_Pretrain_Hindi_FineTune/training_" +  str(i) + "/cp.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)


  #hyperparameter tuning, only choose the best ones found with hindi
  random.seed(42)
  learning_rates = [.001]
  dropouts = [0]
  hidden_sizes = [(256, 128)]
  train_layers = [0]

  count = 0
  for learning_rate in learning_rates:
    for dropout in dropouts:
      for train_layer in train_layers:
        for hidden_size in hidden_sizes:

          count += 1

          reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                              train_layers = train_layer,
                                                              hidden_initializer = 'he_uniform',
                                                              dropout = dropout,
                                                              hidden_sizes = hidden_size
                                                              )

  # Loads the weights
  reloaded_model.load_weights(reload_path)

  early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_f1_m', 
                                                 mode = 'max',
                                                 patience=3, 
                                                 restore_best_weights=True)

  cp_callback_retrain = [early_stopper, tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=0)]


  print('COUNT = {:.0f}'.format(i))


  history = reloaded_model.fit([hindi_x_train.input_ids, hindi_x_train.token_type_ids, hindi_x_train.attention_mask],
                              hindi_y_train,
                              validation_data=([hindi_x_dev.input_ids, hindi_x_dev.token_type_ids, hindi_x_dev.attention_mask],
                              hindi_y_dev),
                                    epochs=15,
                                    batch_size=20,
                                    callbacks=cp_callback_retrain,
                                    verbose = 1
                                    )
    
  # append_record({'Count':i, 'history':history.history})


COUNT = 2
Epoch 1/15
212/212 [==============================] - 219s 994ms/step - loss: 0.6796 - acc: 0.5799 - f1_m: 0.5505 - precision_m: 0.5989 - recall_m: 0.5658 - val_loss: 0.6750 - val_acc: 0.5766 - val_f1_m: 0.4236 - val_precision_m: 0.7435 - val_recall_m: 0.3186
Epoch 2/15
212/212 [==============================] - 212s 1s/step - loss: 0.6601 - acc: 0.6021 - f1_m: 0.5640 - precision_m: 0.6154 - recall_m: 0.5606 - val_loss: 0.6460 - val_acc: 0.6560 - val_f1_m: 0.6038 - val_precision_m: 0.7333 - val_recall_m: 0.5333
Epoch 3/15
212/212 [==============================] - 213s 1s/step - loss: 0.6476 - acc: 0.6267 - f1_m: 0.5889 - precision_m: 0.6293 - recall_m: 0.5910 - val_loss: 0.6377 - val_acc: 0.6238 - val_f1_m: 0.6677 - val_precision_m: 0.6040 - val_recall_m: 0.7651
Epoch 4/15
212/212 [==============================] - 215s 1s/step - loss: 0.6391 - acc: 0.6314 - f1_m: 0.6082 - precision_m: 0.6385 - recall_m: 0.6244 - val_loss: 0.6496 - val_acc: 0.6333 - val_f1_m: 0.4997 - val_pr

#Run korean on pretrained English Models

In [ ]:
count = 0
for i in range(1,4):

  

  #Define reload directory
  if i > 1:
    reload_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English 22_3_31/training_" + str(i) + "_Good/cp.ckpt"
    reload_dir = os.path.dirname(reload_path)

  else:
    reload_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_For_Hindi/cp.ckpt"
    reload_dir = os.path.dirname(reload_path)

  #Define checkpoint directory
  checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English_Pretrain_Korean_FineTune/Dropout-0_15/training_" +  str(i) + "/cp.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)


  #hyperparameter tuning, only choose the best ones found with korean
  random.seed(42)
  learning_rates = [.001]
  dropouts = [0.15]
  hidden_sizes = [(256, 128)]
  train_layers = [0]

  
  for learning_rate in learning_rates:
    for dropout in dropouts:
      for train_layer in train_layers:
        for hidden_size in hidden_sizes:

          count += 1

          if count <= 1:
            continue


          

          reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                              train_layers = train_layer,
                                                              hidden_initializer = 'he_uniform',
                                                              dropout = dropout,
                                                              hidden_sizes = hidden_size
                                                              )

  # Loads the weights
          reloaded_model.load_weights(reload_path)

          early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_f1_m', 
                                                        mode = 'max',
                                                        patience=3, 
                                                        restore_best_weights=True)

          cp_callback_retrain = [early_stopper, tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                                save_weights_only=True,
                                                                verbose=0)]


          print('COUNT = {:.0f}'.format(i))


          history = reloaded_model.fit([korean_x_train.input_ids, korean_x_train.token_type_ids, korean_x_train.attention_mask],
                                      korean_y_train,
                                      validation_data=([korean_x_dev.input_ids, korean_x_dev.token_type_ids, korean_x_dev.attention_mask],
                                      korean_y_dev),
                                            epochs=15,
                                            batch_size=20,
                                            callbacks=cp_callback_retrain,
                                            verbose = 1
                                            )
    
  # append_record({'Count':i, 'history':history.history})


COUNT = 2
Epoch 1/15
292/292 [==============================] - 190s 623ms/step - loss: 0.6720 - acc: 0.5908 - f1_m: 0.5706 - precision_m: 0.6002 - recall_m: 0.6080 - val_loss: 0.6645 - val_acc: 0.6049 - val_f1_m: 0.4771 - val_precision_m: 0.7442 - val_recall_m: 0.3638
Epoch 2/15
292/292 [==============================] - 176s 602ms/step - loss: 0.6325 - acc: 0.6302 - f1_m: 0.6166 - precision_m: 0.6325 - recall_m: 0.6396 - val_loss: 0.6182 - val_acc: 0.6543 - val_f1_m: 0.6482 - val_precision_m: 0.6731 - val_recall_m: 0.6462
Epoch 3/15
292/292 [==============================] - 176s 602ms/step - loss: 0.6220 - acc: 0.6504 - f1_m: 0.6355 - precision_m: 0.6524 - recall_m: 0.6574 - val_loss: 0.6161 - val_acc: 0.6571 - val_f1_m: 0.6512 - val_precision_m: 0.6643 - val_recall_m: 0.6560
Epoch 4/15
292/292 [==============================] - 175s 600ms/step - loss: 0.6213 - acc: 0.6472 - f1_m: 0.6193 - precision_m: 0.6554 - recall_m: 0.6239 - val_loss: 0.6078 - val_acc: 0.6598 - val_f1_m: 0.6503

##Run translated Hindi on Pre-trained English

In [77]:
#Function to add arecord to a json file
def append_record(record):
    with open('/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English_Pretrain_Hindi_Translated_FineTune/settings_history.txt', 'a') as f:
        json.dump(record, f)
        f.write(os.linesep)

In [78]:
for i in range(1,2):

  #Define reload directory
  reload_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_For_Hindi+Korean/cp.ckpt"
  reload_dir = os.path.dirname(reload_path)

  #Define checkpoint directory
  checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English_Pretrain_Hindi_Translated_FineTune/training_" +  str(i) + "/cp.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)


  #hyperparameter tuning, only choose the best ones found with hindi
  random.seed(42)
  learning_rates = [.001]
  dropouts = [0]
  hidden_sizes = [(256, 128)]
  train_layers = [0]

  count = 0
  for learning_rate in learning_rates:
    for dropout in dropouts:
      for train_layer in train_layers:
        for hidden_size in hidden_sizes:

          count += 1

          reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                              train_layers = train_layer,
                                                              hidden_initializer = 'he_uniform',
                                                              dropout = dropout,
                                                              hidden_sizes = hidden_size
                                                              )

  # Loads the weights
  reloaded_model.load_weights(reload_path)

  early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_f1_m', 
                                                 mode = 'max',
                                                 patience=3, 
                                                 restore_best_weights=True)

  cp_callback_retrain = [early_stopper, tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=0)]


  print('COUNT = {:.0f}'.format(i))


  history = reloaded_model.fit([hindi_translated_x_train.input_ids, hindi_translated_x_train.token_type_ids, hindi_translated_x_train.attention_mask],
                              hindi_translated_y_train,
                              validation_data=([hindi_translated_x_dev.input_ids, hindi_translated_x_dev.token_type_ids, hindi_translated_x_dev.attention_mask],
                              hindi_translated_y_dev),
                                    epochs=15,
                                    batch_size=20,
                                    callbacks=cp_callback_retrain,
                                    verbose = 1
                                    )
    
  # append_record({'Count':i, 'history':history.history})


COUNT = 1
Epoch 1/15
212/212 [==============================] - 225s 1s/step - loss: 0.6656 - acc: 0.6064 - f1_m: 0.5669 - precision_m: 0.6202 - recall_m: 0.5753 - val_loss: 0.6326 - val_acc: 0.6560 - val_f1_m: 0.6708 - val_precision_m: 0.6535 - val_recall_m: 0.7155
Epoch 2/15
212/212 [==============================] - 217s 1s/step - loss: 0.6385 - acc: 0.6439 - f1_m: 0.6139 - precision_m: 0.6643 - recall_m: 0.6170 - val_loss: 0.6207 - val_acc: 0.6578 - val_f1_m: 0.6237 - val_precision_m: 0.7122 - val_recall_m: 0.5727
Epoch 3/15
212/212 [==============================] - 215s 1s/step - loss: 0.6238 - acc: 0.6491 - f1_m: 0.6129 - precision_m: 0.6612 - recall_m: 0.6201 - val_loss: 0.6079 - val_acc: 0.6786 - val_f1_m: 0.6613 - val_precision_m: 0.7157 - val_recall_m: 0.6328
Epoch 4/15
212/212 [==============================] - 217s 1s/step - loss: 0.6092 - acc: 0.6640 - f1_m: 0.6406 - precision_m: 0.6691 - recall_m: 0.6515 - val_loss: 0.6172 - val_acc: 0.6730 - val_f1_m: 0.7087 - val_preci

##Run translated korean on Pretrain English

In [ ]:
#Function to add arecord to a json file
def append_record(record):
    with open('/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English_Pretrain_Korean_Translated_FineTune/settings_history.txt', 'a') as f:
        json.dump(record, f)
        f.write(os.linesep)

In [81]:
for i in range(1,2):

  #Define reload directory
  reload_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_For_Hindi+Korean/cp.ckpt"
  reload_dir = os.path.dirname(reload_path)

  #Define checkpoint directory
  checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/Models_English_Pretrain_Korean_Translated_FineTune/training_" +  str(i) + "/cp.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)


  #hyperparameter tuning, only choose the best ones found with hindi
  random.seed(42)
  learning_rates = [.001]
  dropouts = [0.15]
  hidden_sizes = [(256, 128)]
  train_layers = [0]

  count = 0
  for learning_rate in learning_rates:
    for dropout in dropouts:
      for train_layer in train_layers:
        for hidden_size in hidden_sizes:

          count += 1

          reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                              train_layers = train_layer,
                                                              hidden_initializer = 'he_uniform',
                                                              dropout = dropout,
                                                              hidden_sizes = hidden_size
                                                              )

  # Loads the weights
  reloaded_model.load_weights(reload_path)

  early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_f1_m', 
                                                 mode = 'max',
                                                 patience=3, 
                                                 restore_best_weights=True)

  cp_callback_retrain = [early_stopper, tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=0)]


  print('COUNT = {:.0f}'.format(i))


  history = reloaded_model.fit([korean_translated_x_train.input_ids, korean_translated_x_train.token_type_ids, korean_translated_x_train.attention_mask],
                              korean_translated_y_train,
                              validation_data=([korean_translated_x_dev.input_ids, korean_translated_x_dev.token_type_ids, korean_translated_x_dev.attention_mask],
                              korean_translated_y_dev),
                                    epochs=15,
                                    batch_size=20,
                                    callbacks=cp_callback_retrain,
                                    verbose = 1
                                    )
    
  # append_record({'Count':i, 'history':history.history})
#.7003 w/o dropout

COUNT = 1
Epoch 1/15
292/292 [==============================] - 305s 1s/step - loss: 0.6564 - acc: 0.6148 - f1_m: 0.6043 - precision_m: 0.6186 - recall_m: 0.6426 - val_loss: 0.6384 - val_acc: 0.6379 - val_f1_m: 0.5847 - val_precision_m: 0.6964 - val_recall_m: 0.5316
Epoch 2/15
292/292 [==============================] - 297s 1s/step - loss: 0.6378 - acc: 0.6333 - f1_m: 0.6223 - precision_m: 0.6297 - recall_m: 0.6520 - val_loss: 0.6149 - val_acc: 0.6571 - val_f1_m: 0.6783 - val_precision_m: 0.6527 - val_recall_m: 0.7283
Epoch 3/15
292/292 [==============================] - 295s 1s/step - loss: 0.6269 - acc: 0.6405 - f1_m: 0.6344 - precision_m: 0.6361 - recall_m: 0.6654 - val_loss: 0.6131 - val_acc: 0.6749 - val_f1_m: 0.6951 - val_precision_m: 0.6708 - val_recall_m: 0.7470
Epoch 4/15
292/292 [==============================] - 295s 1s/step - loss: 0.6309 - acc: 0.6453 - f1_m: 0.6300 - precision_m: 0.6435 - recall_m: 0.6646 - val_loss: 0.6157 - val_acc: 0.6475 - val_f1_m: 0.6948 - val_preci

#Previous Code (Extra)

In [ ]:
Analysis = pd.DataFrame(data = {'predicted_prob':predictions.reshape(529,), 'predicted_class':(predictions>0.5).reshape(529).astype(int),'true_class':hindi_y_dev})

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(list(Analysis.true_class), list(Analysis.predicted_class), labels=[0, 1])
print(confusion_matrix)

In [ ]:
Analysis['correct_prediction'] = Analysis.predicted_class == Analysis.true_class

In [ ]:
Analysis.reset_index(drop=True, inplace=True)

In [ ]:
# Correctly classified as toxic with the largest probability
pd.set_option('display.max_colwidth', None)

temp_df = Analysis[(Analysis.correct_prediction==True) & (Analysis.true_class == 1)].nlargest(25, ['predicted_prob'])

pd.concat([hindi_df_dev.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

In [ ]:
# Correctly classified as non-toxic with the lowest probability (of toxic)
pd.set_option('display.max_colwidth', None)

temp_df = Analysis[(Analysis.correct_prediction==True) & (Analysis.true_class == 0)].nsmallest(25, ['predicted_prob'])

pd.concat([hindi_df_dev.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

In [ ]:
# incorrectly classified as toxic with the highest probability (of toxic)
pd.set_option('display.max_colwidth', None)


temp_df = Analysis[(Analysis.correct_prediction==False) & (Analysis.true_class == 0)].nlargest(25, ['predicted_prob'])

pd.concat([hindi_df_dev.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

In [ ]:
# incorrectly classified as non-toxic with the lowest probability (of toxic)
pd.set_option('display.max_colwidth', None)


temp_df = Analysis[(Analysis.correct_prediction==False) & (Analysis.true_class == 1)].nsmallest(25, ['predicted_prob'])

pd.concat([hindi_df_dev.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

In [ ]:
# Model is unsure, predicted prob is abs(predicted_prob - 0.5)

pd.set_option('display.max_colwidth', None)


temp_df = Analysis.predicted_prob.sub(0.5).abs().nsmallest(25)

pd.concat([hindi_df_dev.iloc[list(temp_df.index)], temp_df], axis = 1).rename(columns={'predicted_prob':'abs(predicted_prob - 0.5)'})


In [ ]:
outputs = mbert_model(hindi_x_dev.input_ids[:5], hindi_x_dev.token_type_ids[:5], hindi_x_dev.attention_mask[:5], output_attentions=True )

In [ ]:
outputs